# Euler Angles, rotation Matrixes and Quarternions


In [ ]:
import os
import numpy as np
import open3d as o3d
import math

### Function that returns rotation matrix given angles alpha, beta and gamma (in radians) 

In [ ]:
# alpha -> angle of rotation about z-axis
# beta -> angle of rotation about y-axis
# gamma -> angle of rotation about x-axis
def rotMatrix(gamma, beta, alpha):
    matrix = []
    row1 = []
    row1.append(math.cos(beta) * math.cos(alpha))
    row1.append(math.sin(gamma)*math.sin(beta)*math.cos(alpha) - math.cos(gamma)*math.sin(alpha))
    row1.append(math.cos(gamma)*math.sin(beta)*math.cos(alpha) + math.sin(gamma)*math.sin(alpha))
    matrix.append(row1)
    row2 = []
    row2.append(math.cos(beta) * math.sin(alpha))
    row2.append(math.sin(gamma)*math.sin(beta)*math.sin(alpha) + math.cos(gamma)*math.cos(alpha))
    row2.append(math.cos(gamma)*math.sin(beta)*math.sin(alpha) - math.sin(gamma)*math.cos(alpha))
    matrix.append(row2)
    row3 = []
    row3.append(-math.sin(beta))
    row3.append(math.sin(gamma)*math.cos(beta))
    row3.append(math.cos(gamma)*math.cos(beta))
    matrix.append(row3)
    return matrix



### Function that converts a given quarternion to a rotation matrix

In [ ]:
def QTM(qx, qy, qz, qw):
    #  qw is the real part of the quarternion
    matrix = []
    row1 = []
    row1.append(1 - 2 * qy**2 - 2 * qz**2)
    row1.append(2 * qx * qy + 2 * qz * qw)
    row1.append(2 * qx * qz - 2 * qy * qw)
    matrix.append(row1)
    row2 = []
    row2.append(2 * qx * qy - 2 * qz * qw)
    row2.append(1 - 2 * qx**2 - 2 * qz**2)
    row2.append(2 * qy * qz + 2 * qx * qw)
    matrix.append(row2)
    row3 = []
    row3.append(2 * qx * qz + 2 * qy * qw)
    row3.append(2 * qy * qz - 2 * qx * qw)
    row3.append(1 - 2 * qx**2 - 2 * qy**2)
    matrix.append(row3)
    return matrix

# Each vector is expressed as a row here so that a vector is rotated by post multiplying it with a rotation matrix.


### Function that converts a given rotation matrix to a quarternion

In [ ]:
def MTQ(m):
    ans = []
    if m[2][2] < 0:
        if m[0][0] > m[1][1]:
            trace = 1 + m[0][0] - m[1][1] - m[2][2]
            ans.append(trace)
            ans.append(m[0][1] + m[1][0])
            ans.append(m[2][0] + m[0][2])
            ans.append(m[1][2] - m[2][1])
        else:
            trace = 1 - m[0][0] + m[1][1] - m[2][2]
            ans.append(m[0][1] + m[1][0])
            ans.append(trace)
            ans.append(m[1][2] + m[2][1])
            ans.append(m[2][0] - m[0][2])
    else:
        if m[0][0] < -m[1][1]:
            trace = 1 - m[0][0] - m[1][1] + m[2][2]
            ans.append(m[0][2] + m[2][0])
            ans.append(m[1][2] + m[2][1])
            ans.append(trace)
            ans.append(m[0][1] - m[1][0])
        else:
            trace = 1 + m[0][0] + m[1][1] + m[2][2]
            ans.append(m[1][2] - m[2][1])
            ans.append(m[2][0] - m[0][2])
            ans.append(m[0][1] - m[1][0])
            ans.append(trace)
    ans = np.array(ans)
    ans = ans / math.sqrt(trace) * 2


## Gimbal Lock

- We can achieve the desired orientation using Euler rotations by combining multiple euler transformations. However, the rotations about different axes depend upon each other. We can say that transformations can change the asis of coordinate system. 

- In some situations, two of the axes can line up perfectly. In this case, rotating two of the axes has the same effect. In totality, we are left with only 2 dimensions of movement and we have lost a degree of freedom. This locking type of phenomenon is know as the Gimbal Lock.

### Rotation matrix for airplane increasing pitch by 90 degreees

For $ \beta = \pi / 2$ 

$R_{gmb \beta}$ = $\begin{bmatrix} 1 & 0 & 0 \\ 0 & cos(\gamma) & -sin(\gamma) \\ 0 & sin(\gamma) & cos(\gamma) \end{bmatrix}$ $\begin{bmatrix} 0 & 0 & 1 \\ 0 & 1 & 0 \\ -1 & 0 & 0 \end{bmatrix}$ $\begin{bmatrix} cos(\alpha) & -sin(\alpha) & 0 \\ sin(\alpha) & cos(\alpha) & 0 \\ 0 & 0 & 1 \end{bmatrix}$ 

$R_{gmb \beta}$ = $\begin{bmatrix} 0 & 0 & 1 \\ cos(\gamma)sin(\alpha) + cos(\alpha)sin(\gamma) & cos(\gamma)cos(\alpha) - sin(\alpha)sin(\gamma) & 0 \\ -cos(\gamma)cos(\alpha) + sin(\alpha)sin(\gamma) &  cos(\gamma)sin(\alpha) + cos(\alpha)sin(\gamma) & 0 \end{bmatrix}$



$P_{new} = R_{gmb \beta} * P$ = $\begin{bmatrix} 0 & 0 & 1 \\ cos(\gamma)sin(\alpha) + cos(\alpha)sin(\gamma) & cos(\gamma)cos(\alpha) - sin(\alpha)sin(\gamma) & 0 \\ -cos(\gamma)cos(\alpha) + sin(\alpha)sin(\gamma) &  cos(\gamma)sin(\alpha) + cos(\alpha)sin(\gamma) & 0 \end{bmatrix}$ $\begin{bmatrix} 0 \\ 1  \\ 0  \end{bmatrix}$

= $\begin{bmatrix} 0 \\ cos(\gamma)cos(\alpha) - sin(\alpha)sin(\gamma) \\ cos(\gamma)sin(\alpha) + cos(\alpha)sin(\gamma) \end{bmatrix}$

= $\begin{bmatrix} 0 \\ sin(\alpha + \gamma) \\ cos(\alpha + \gamma) \end{bmatrix}$


1. $P_{new} = [1, 0, 0]ᵀ $ is not possibleas the first entry of $P_{new}$ has to be zero

2. If $P_{new}$ lies on the XY unit cicle, then, it's x coordinate will be zero and the sum of squares of the x and y coordinats will be 1. However, $P_{new}$ has to have x coordinate as 0. Therefore, y coordinate has to be +1 or -1. Therefore $P_{new} = [0, +1, 0]ᵀ$ and $P = [0, -1, 0]ᵀ$ are two posible solutions. Now, if $ cos(\alpha + \gamma) = 0 $ and $ sin(\alpha + \gamma)  = +1 , -1 $, then this gives us infinte solutions for $ \alpha + \gamma $ where their sum is either $\pi/2$ or $-\pi/2$.

3. If $P_{new}$ lies on the YZ unit cicle, then for any y belonging to {0,1}, we can get an $z^2$ = {1 - $y^2$}. Therefore, wehave infiite possibilities for $P_{new}$. Now, $cos(\alpha + \gamma)^2$ + $sin(\alpha + \gamma)^2$ is always 1 which is already satisfied by the equation: $z^2$ = 1 - $y^2$. Therefore, we will be able to get $ \alpha,  \gamma $ for all these solutions. 

- The value of 90 degrees as $ \beta $ is a problematic value because it results in the isue of gimbal lock.